In [1]:
import requests
page = requests.get("https://www.alko.fi/tuotteet/909527")
page.content

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')


print(soup.prettify())

<!DOCTYPE doctype html>
<html lang="fi">
 <head>
  <script>
   requestManifest();
function requestManifest() {
var xmlHttp = new XMLHttpRequest();
xmlHttp.open("GET", "/INTERSHOP/rest/WFS/Alko-OnlineShop-Site/-/storejson/manifest.json", false); // false for synchronous request
xmlHttp.send(null);
xmlHttp.onreadystatechange = manifestRequestResponseReceived(xmlHttp);
}
function manifestRequestResponseReceived(xmlHttp) {
var manifestExists = xmlHttp.responseText;
if ( manifestExists == 'true' ) {
var link = document.createElement('link');
link.rel = "manifest";
link.href = "/INTERSHOP/static/BOS/Alko-Site/OnlineShop/Alko-OnlineShop/en_US/mobile-app/manifest.json";
document.head.appendChild(link);
}
}
  </script>
  <meta content="IE=edge" http-equiv="x-ua-compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="2050" http-equiv="refresh">
   <meta content="1800" http-equiv="refresh">
    <meta content="Bollinger R.D. 

In [2]:
taste_description=soup.find_all('div', class_="taste-description")
bag=taste_description[0].get_text()


product_name=soup.find_all('h1', class_="product-name")
name=product_name[0].get_text()

volume=soup.find_all('span', class_="volume")
volume=volume[0].get_text()

In [3]:
import re
bag=re.sub("\n","",bag)
list_of_tags=re.split(",",bag)
list_of_tags2=[]
for element in list_of_tags:
    list_of_tags2.append((element.lstrip()).rstrip().lower())
print(list_of_tags2)

['erittäin kuiva', 'erittäin hapokas', 'sitruksinen', 'paahteinen', 'aprikoosinen']


In [113]:
def getTasteProfile(soup):
    '''extract the taste profile and name from html'''
    #taste profile
    taste_description=soup.find_all('div', class_="taste-description")
    bag=taste_description[0].get_text()
    #extract name
    product_name=soup.find_all('h1', class_="product-name")
    name=product_name[0].get_text()
    #extraxt product volume (packaging size)
    volume=soup.find_all('span', class_="volume")
    volume=volume[0].get_text()
    volume=re.sub("\xa0l","",volume)
    # some volume fields contain text
    #volume=float(volume)
    
    bag=re.sub("\n","",bag)
    
    list_of_tags=re.split(",",bag)
    list_of_tags2=[]
    for element in list_of_tags:
        list_of_tags2.append((element.lstrip()).rstrip().lower())
    return list_of_tags2, name, volume

In [5]:
getTasteProfile(soup)

(['erittäin kuiva',
  'erittäin hapokas',
  'sitruksinen',
  'paahteinen',
  'aprikoosinen'],
 'Bollinger R.D. Extra Brut 2004',
 0.75)

In [107]:
len(soup)

766

In [129]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Oton ensimmäinen "commit" 3.3.2019

def getTuotetiedot(soup):

	label_raw = soup.find_all("div", {"class": "column tiny-6 small-b1 h8 fact-label"})
	value_raw = soup.find_all("div", {"class": "column tiny-6 small-h6 fact-data"})

	label = [tag.text for tag in label_raw]
	value = [tag.text.strip() for tag in value_raw]
    
	df = pd.DataFrame({"LABEL": label, "VALUE": value})

	return df

def getTags(soup):

	try:
		wine_tags_raw = soup.find("div", {"class": "product-tags-wrap pdp-wine-tags"}).find_all("div", {"class": "tooltip-heading"})
		wine_tags = [tag.text.strip() for tag in wine_tags_raw]
	except:
		wine_tags = []

	try:
		region_tags_raw = soup.find("div", {"class": "product-tags-wrap pdp-region-tags"}).find_all("div", {"class": "tooltip-heading"})
		region_tags = [tag.text.strip() for tag in region_tags_raw]
	except:
		pass

	try:
		product_tags_raw = soup.find("div", {"class": "product-tags-wrap pdp-region-tags"}).find_all("a", {"class": "tag-not-clickable"})
		product_tags = [tag.text.strip() for tag in product_tags_raw]
	except:
		pass

	#region = region_tags + product_tags

	return wine_tags, region_tags[0],region_tags[1:], product_tags

def getIcons(soup):
    
    try:
		icons_raw = soup.find("ul", {"class": "food-pairings"}).find_all("a", {"class": "trackable-filter-item pdp-symbol-link"})
    except:
		pass

	icons = []

	for tag in icons_raw:

		s = str(tag)
		t = re.search('filterValue":"(.+?)"', s).group(1)

		icons.append(t)

	return icons


In [99]:
def getProductCategory(soup):
    try:
        wine_tags_raw = soup.find("a", {"class": "product-category trackable-filter-item"}).get_text()
    except:
        wine_tags_raw = []
    return wine_tags_raw.strip()

def getProductCharacteristics(soup):
    try:
        characteristics_raw = soup.find("div", {"class": "taste-type-text"}).get_text()
    except:
        characteristics_raw = []
    return characteristics_raw.strip()

getProductCharacteristics(soup)


'Nuanced & structured'

In [99]:
# script to scrape product numbers

link_start="https://www.alko.fi/tuotteet/tuotelistaus?SearchTerm=*&PageSize=12&SortingAttribute=&PageNumber="
# input page number in between these two strings
link_end="&SearchParameter=%26%40QueryTerm%3D*%26ContextCategoryUUID%3D6Q7AqG5uhTIAAAFVOmkI2BYA%26OnlineFlag%3D1"
pages=[]
for i in range(1,100):
    link=link_start+str(i)+link_end
    try:
        page = requests.get(link)
        pages.append(page)
    except:
        # ignore the error
        break
        
# end with pages object with list of pages

def extract_product_num_from_line(string):
    '''function to extract product name from html string'''
    result = re.search('data-availability="(.*)"></span>', string)
    return result.group(1)

product_numbers=[]
for page in pages:
    soup = BeautifulSoup(page.content, 'html.parser')
    product_number_lines=soup.find_all('span', class_="product-availability-symbol")
    for line in product_number_lines:
        product_numbers.append(extract_product_num_from_line(str(line)))

unique_product_numbers=list(set(product_numbers))
len(unique_product_numbers)

8525

In [100]:
file = open('product_numbers.txt','w') 
for num in unique_product_numbers:
    file.write(str(num)+",") 
file.close() 

In [174]:
# wrapper function for all data
def product_data_wrapper(soup):
    # get all data to variables
    tasteprofile=getTasteProfile(soup)
    icons=getIcons(soup)
    tags=getTags(soup)
    productinfo=getTuotetiedot(soup)

    productinfo_T=(productinfo.T).reset_index(drop=True)

    # create new dataframe (initialization)
    product_df=productinfo_T
    headers = product_df.iloc[0]
    product_df  = pd.DataFrame(product_df.values[1:], columns=headers)

    # add the rest to the new dataframe
    taste_profile, name, volume=getTasteProfile(soup)
    product_df["TASTE PROFILE"]=str(','.join(map(str, taste_profile)))
    product_df["NAME"]=str(name)
    product_df["VOLUME"]=str(volume)
    icons=getIcons(soup)
    product_df["ICONS"]=str(','.join(map(str, icons)))
    grape, country, region1, region2=getTags(soup)
    #product_df["TAGS"]=str(getTags(soup))
    product_df["COUNTRY"]=str(country)
    try:
        product_df["GRAPE"]=str(','.join(map(str, grape)))
    except:
        pass
    try:
        product_df["REGION1"]=str(region1[0])
    except:
        pass
    try:
        product_df["REGION2"]=str(region2[0])
    except:
        pass
    product_df["CHARACTERISTICS"]=str(getProductCharacteristics(soup))
    product_df["CATEGORY"]=str(getProductCategory(soup))
    return product_df

###################
# test the function
prod_num_test=909527
link_start="https://www.alko.fi/en/tuotteet/" # + add product number
# read page contents to soup object
page = requests.get(link_start+str(prod_num_test))
soup = BeautifulSoup(page.content, 'html.parser')
print(product_data_wrapper(soup))

0 PRODUCT NUMBER ALCOHOL    SUGAR     ACIDS             ENERGY PRICE / LITRE  \
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   

0 PACKAGING       CLOSURE             PRODUCER  SUPPLIER  \
0    bottle  natural cork  Champagne Bollinger  Wennerco   

0             ...              \
0             ...               

0                                      TASTE PROFILE  \
0  brut,very acidic,citrus notes,toasted,roasted,...   

0                            NAME VOLUME  \
0  Bollinger R.D. Extra Brut 2004   0.75   

0                                              ICONS COUNTRY  \
0  meditation beverage,game birds,clams, mussels,...  France   

0                  GRAPE    REGION1       REGION2    CHARACTERISTICS  \
0  Pinot Noir,Chardonnay  Champagne  AC Champagne  Generous & toasty   

0                    CATEGORY  
0  Sparkling wine & champagne  

[1 rows x 21 columns]


In [172]:


for i in range(0,10):
    product_df=product_data_wrapper(soup)
    if 'all_products' in locals():
        all_products=all_products.append(product_df)
    else:
        all_products=product_df

print(all_products)

0 PRODUCT NUMBER ALCOHOL    SUGAR     ACIDS             ENERGY PRICE / LITRE  \
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   
0         909527   12.0%  3.0 g/l  86.0 g/l  70.0 Kcal / 100ml    253.19 EUR   

0 PACKAGING       CLOSURE             PRODUCER  SUPPLIER  \
0    bottle  natural cork  Champagne Bollinger  Wennerco   

In [44]:
# load product numbers from file
import csv
del unique_product_numbers
if 'unique_product_numbers' in locals():
    1+1
else:
    with open('product_numbers.txt') as csvfile:
        unique_product_numbers=csvfile.read()
unique_product_numbers=re.split(',',unique_product_numbers)
len(unique_product_numbers)

8526

In [183]:
import time

# loop through product pages and load the data
soups=[]
try:
    del all_products
except:
    pass

start_t=time.time()
counter=1
error_producing=[]
for product_number in unique_product_numbers[:100]:
    # generate www address
    link_start="https://www.alko.fi/en/tuotteet/" # + add product number
    
    # read page contents to soup object
    page = requests.get(link_start+str(product_number))
    soup = BeautifulSoup(page.content, 'html.parser')
    soups.append(soup)
    # translate soup object to a dataframe
    # some products cause errors
    # typically these are non-beverage products
    try:
        product_df=product_data_wrapper(soup)
    except:
        error_producing.append(product_number)
        
    if 'all_products' in locals():
        all_products=all_products.append(product_df)
    else:
        all_products=product_df
    counter=counter+1
    if counter%100==0:
        print(str(counter)+", "+str(time.time()-start_t))
end_t=time.time()
print("time elapsed: "+str(end_t-start_t))
print("time per page: "+str((end_t-start_t)/counter))
print("expected duration: "+str(((end_t-start_t)/counter)*len(unique_product_numbers)/60)+" mins")
all_products.head()

/Users/Heppa/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


100, 265.6406762599945
time elapsed: 268.33088207244873
time per page: 2.656741406657908
expected duration: 377.5229538860887 mins


,ACIDS,ADDITIONAL INFORMATION,ALCOHOL,BEER TYPE,BITTERNESS,CATEGORY,CHARACTERISTICS,CLOSURE,COUNTRY,DEPOSIT,...,PRODUCER,PRODUCT NUMBER,REGION1,REGION2,SELECTION,SUGAR,SUPPLIER,TASTE PROFILE,VOLUME,WORT STRENGTH
0,4.0 g/l,NaN,15.0%,NaN,NaN,Dessert wine & other fortified wines,Sherry,natural cork,Spain,0.1 €,...,González Byass,003572,NaN,DO Jerez-Xérès-Sherry,general selection,NaN,Uniq Drinks Finland,"green-yellow,dry,citrus notes,almond notes,lig...",0.75,NaN
0,4.6 g/l,NaN,14.5%,NaN,NaN,Red wine,Luscious & jammy,natural cork,United States,0.1 €,...,Hahn Family Wines,479657,California,Monterey,general selection,4.0 g/l,Amka,"full-bodied,medium tannic,cherry notes,wild st...",0.75,NaN
0,9.4 g/l,NaN,12.0%,NaN,NaN,Sparkling wine & champagne,Nuanced & structured,natural cork,France,NaN,...,Champagne Laurent Lequart,943447,Champagne,AC Champagne,sale-to-order selection,3.0 g/l,Vita Vinum,"brut,very acidic,citrus notes,ripe plum notes,...",0.75,NaN
0,3.9 g/l,NaN,6.0%,NaN,NaN,Cider,British style apple cider,crown cap,England,0.1 €,...,Weston & Sons,774166,NaN,NaN,general selection,30.0 g/l,Servaali,"dry,golden-yellow,medium acidic,apple notes,wh...",0.5,NaN
0,5.6 g/l,NaN,12.5%,NaN,NaN,White wine,Nuanced & structured,natural cork,Portugal,NaN,...,Wine & Soul,916567,NaN,DOC Douro,sale-to-order selection,NaN,Handpicked Wines Finland Ab,"dry,acidic,grapefruity,mineral notes,minerally",0.75,NaN


In [181]:
product_number

'833300'